In [ ]:
import pandas as pd
import requests
import time
from datetime import date,datetime

pd.set_option('display.max_columns', None)
import os

from utils import *


In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
token = os.getenv('data_access_token')
client_id = int(os.getenv('client_id'))
business_timezone_offset = int(os.getenv('business_timezone_offset'))



# Branches

In [ ]:
last_page = call_foodics('branches', 1, client_id, token, return_last_page=True)

list_responses = call_foodics('branches', last_page, client_id, token)

In [ ]:
df_branches = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_branches['client_id'] = client_id
df_branches['slug'] = df_branches.name.apply(lambda x: generate_slug(x))
df_branches

In [ ]:
df_branches = df_branches[['id', 'client_id', 'name', 'slug', 'opening_from', 'opening_to', 'created_at', 'updated_at', 'deleted_at']]

In [ ]:
df_branches

# Products

In [ ]:
last_page = call_foodics('products', 1, client_id,token, return_last_page=True)

list_responses = call_foodics('products', last_page, client_id,token, includables='category')

In [ ]:
df_products = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_products['client_id'] = client_id
df_products['slug'] = df_products.name.apply(lambda x: generate_slug(x))

In [ ]:
df_products.head()

In [ ]:
category_name_col = os.getenv('category_name_col')
print(category_name_col)

df_products['category_name'] = df_products['category'].apply(lambda x: x[category_name_col])
df_products = df_products[~df_products.category_name.isna()]
df_products['category_name'] = df_products['category_name'].apply(lambda x: x.capitalize())

df_products['category_id'] = df_products['category'].apply(lambda x: x['id'])
df_products.rename(columns={'sku':"sku"}, inplace=True)
df_products.drop('category', axis=1, inplace=True)
# df_products = df_products[df_products.deleted_at.isna()]
df_products.head()

In [ ]:
df_products = df_products[['id', 'client_id', 'sku', 'slug', 'name', 'category_id', 'is_active', 'is_stock_product', 'price', 'created_at', 'updated_at', 'deleted_at']]

In [ ]:

df_products = df_products.rename(columns={'is_stock_product':"is_stock"})

In [ ]:
df_products.head()

# Categories

In [ ]:
last_page = call_foodics('categories', 1, client_id,token, return_last_page=True)


list_responses = call_foodics('categories', last_page, client_id, token,) 

In [ ]:
df_categories = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_categories['client_id'] = client_id
df_categories['slug'] = df_categories.name.apply(lambda x: generate_slug(x))
df_categories.head()

In [ ]:
cats_to_be_deleted = df_categories[~df_categories.deleted_at.isna()]
cats_to_be_deleted.head()

In [ ]:
df_categories = df_categories[df_categories.deleted_at.isna()]

In [ ]:
df_categories = df_categories[['id', 'client_id', category_name_col, 'slug', 'created_at', 'updated_at', 'deleted_at']].rename(columns={category_name_col:"name"})


In [ ]:
df_categories

In [ ]:
df_categories.name = df_categories.name.apply(lambda x: x.capitalize())

In [ ]:
print(df_categories['name'].tolist())

# Orders

In [ ]:
filter = {}
path = f'../../data/{client_id}/raw/orders_final_include.csv'


if os.path.exists(path):
    print('orders_final_include.csv already exists')
    df_orders = pd.read_csv(path)
    df_orders.reset_index(drop=True, inplace=True)  
else:
    print('orders_final_include.csv does not exist')
    includables = 'products,branch,products.product,tags,products.promotion'
    # call the foodics api to get the orders
    last_page = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter, return_last_page=True)
    print('last_page: ', last_page)
    
    list_responses = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter)
    df_orders = pd.DataFrame([item for sublist in list_responses for item in sublist])
    df_orders.to_csv(path, index=False)

# TODO: remove this line after testing
df_orders = pd.read_csv(path)
df_orders.reset_index(drop=True, inplace=True)
    

In [ ]:
df_orders.created_at = pd.to_datetime(df_orders.created_at)
max_date = df_orders.created_at.max().date().strftime('%Y-%m-%d')
max_date

In [ ]:
today = date.today().strftime('%Y-%m-%d')
today

In [ ]:
df_orders=df_orders.reset_index(drop=True)

In [ ]:
df_orders.head()

In [ ]:
df_orders['branch'] = df_orders['branch'].astype(str)
df_orders['tags'] = df_orders['tags'].astype(str)
df_orders['payments'] = df_orders['payments'].astype(str)
df_orders['products'] = df_orders['products'].astype(str)


In [ ]:
df_orders['branch_id'] = df_orders['branch'].apply(lambda x: eval(x)['id'])
df_orders['client_id'] = client_id

In [ ]:
orders_header = df_orders[['id', 'client_id', 'branch_id',  'source', 'type', 'status', 'total_price', 'created_at', 'updated_at']]
orders_header.isnull().sum()

In [ ]:
orders_header.head()

In [ ]:
orders_header['ordered_at'] = orders_header['created_at']

# Order Details

In [ ]:
orders_header.head()

In [ ]:
df_orders.head()

In [ ]:
# I want to create a dataframe with the following columns:
# order_id, product_id, category_id, quantity, price

# I will create a list of dictionaries, where each dictionary is a row in the dataframe
# I will then convert the list of dictionaries to a dataframe
import uuid

need_unavailable_category = False

list_order_details = []
length_orders = len(df_orders)
for index, row in df_orders.iterrows():

    order_header_id = row['id']
    branch_id = eval(row['branch'])['id']
    created_at = row['created_at']
    updated_at = row['updated_at']
    total_price = row['total_price']


    for order_product in eval(row['products']):
        order_details_id = str(uuid.uuid4())


        product_id = order_product['product']['id']
        try:
            category_id = df_products[df_products['id'] == product_id]['category_id'].values[0]
        except:
            need_unavailable_category = True
            category_id = '00000000-0000-0000-0000-000000000000'
        
        quantity = order_product['quantity']
        price = order_product['product']['price']

        list_order_details.append({
                                   'id':order_details_id,
                                   'header_id':order_header_id,
                                   'product_id':product_id,
                                   'category_id':category_id,
                                   'client_id':client_id, 
                                   'quantity':quantity,
                                   'price':price,
                                   'created_at':created_at,
                                   'updated_at':updated_at})


In [ ]:
if need_unavailable_category:
    print("Yes, we need to add the unavailable category, make sure to add it in the .env")
    new_row = pd.DataFrame({'id': '00000000-0000-0000-0000-000000000000', 'name': 'Not Available', 'slug':"not-available", 'client_id':client_id}, index=[df_categories.index.max()+1])
    today = date.today().strftime('%Y-%m-%d')
    df_categories = pd.concat([df_categories, new_row], ignore_index=True)
    df_categories.created_at = df_categories.created_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.updated_at = df_categories.updated_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.deleted_at = df_categories.deleted_at.fillna(method='ffill')



df_categories

In [ ]:
orders_details = pd.DataFrame(list_order_details)

In [ ]:
orders_details.head()

In [ ]:
orders_details[orders_details.category_id.isna()].sort_values('created_at')

In [ ]:
cats_to_be_deleted.id.values.tolist()

In [ ]:
order_header_id_to_be_deleted = orders_details[orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())].header_id.unique()

In [ ]:
orders_details = orders_details[~orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())]

In [ ]:
orders_header = orders_header[~orders_header.id.isin(order_header_id_to_be_deleted)]

In [ ]:
df_products = df_products[~df_products.category_id.isin(cats_to_be_deleted.id.values.tolist())]

# UTC TO AMMAN

In [ ]:
orders_header.head()

In [ ]:
orders_details.head()

In [ ]:
# import pytz
from datetime import timedelta

# orders_details['ordered_at'] = pd.to_datetime(orders_details['order_date'])
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_localize('UTC')
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])
# orders_header['order_date'] = orders_header['order_date'].dt.tz_localize('UTC')
# orders_header['order_date'] = orders_header['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

time_difference = timedelta(hours=business_timezone_offset)

orders_header['ordered_at'] += time_difference
# orders_details['order_date'] += time_difference

In [ ]:
orders_header.head()

# Write to CSV

In [ ]:
df_products.to_csv(f'../../data/{client_id}/raw/products.csv', index=False)

In [ ]:
orders_details.to_csv(f'../../data/{client_id}/raw/order_details.csv', index=False)

In [ ]:
orders_header.to_csv(f'../../data/{client_id}/raw/order_header.csv', index=False)

In [ ]:
df_categories.to_csv(f'../../data/{client_id}/raw/categories.csv', index=False)

In [ ]:
df_branches.to_csv(f'../../data/{client_id}/raw/branches.csv', index=False)

# Display all

In [ ]:
df_products.head()

In [ ]:
orders_details.head()

In [ ]:
orders_header.head()

In [ ]:
df_branches.head()

In [ ]:
df_categories